In [4]:
# within coin_flips.txt contains data of the outcomes 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
df = pd.read_csv('coin_flips.txt', header=None)


In [6]:
# Three coins A,B,C with head-prob p_A, p_B, p_C can be chosen for each of N sessions. Once a coin is chosen for the session, that coin is flipped D times.
# For D=20 and N=50, and fixed non-uniform selection coin probabilities pi_A, pi_B, pi_C, which sum to 1, we have this outcome , each row corresponds to a session with 20 binary 1=head 0=tail.


# Compute the probabilities to select each coin to session (3 mixture "pi" probabilities), and also the bias probabilities (3 param "p" probabilities).

# HINT: for each session, since the D flips are independent of each other, what matters is the number of heads out of the batch size D. If chance of head is p_ for each flip, then probability of observing x heads is binomial(x|p_, D). Here is a technical brief note .
# In English: binomial(X|p_,D) = probability to get x heads out of D coin flips, if coin has head-bias p_.

stats.binom.pmf(5, 20, 0.5) # probability of getting 5 heads out of 20 flips, if coin has head-bias 0.5

0.01478576660156255